In [1]:
%matplotlib inline

In [2]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from model import *
from adai_optim import *

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

==> Preparing data..


In [5]:
def optimizers(net, opti_name, lr, weight_decay):
    if opti_name == 'SGD':
        return optim.SGD(net.parameters(), lr=lr, momentum=0, weight_decay=weight_decay)
    elif opti_name == 'SGDM':
        return optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay, nesterov=False)
    elif opti_name == 'Adam':
        return optim.Adam(net.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=weight_decay)
    elif opti_name == 'AMSGrad':
        return optim.Adam(net.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=weight_decay,amsgrad=True)
    elif opti_name == 'AdamW':
        return optim.AdamW(net.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.1)
    elif opti_name == 'Adai':
        return Adai(net.parameters(), lr=lr, betas=(0.1, 0.99), eps=1e-03, weight_decay=weight_decay)
    else:
        raise('Unspecified optimizer.')
    

In [6]:
criterion = nn.CrossEntropyLoss(reduction='mean')

In [7]:
def train(net, optimizer, epoch):
    print('Epoch: %d' % (epoch+1))
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    learning_rate = 0.1
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * targets.size(0)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    print("Training Loss: ", train_loss/total)
    print("Training error:", 1-correct/total)
    return 1 - correct/total, train_loss/total

def test(net):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    print("Test error:", 1-correct/total)
    return 1 - correct/total

In [8]:
def define_models(model):
    if model == 'resnet18':
        return ResNet18()
    elif model == 'vgg16':
        return VGG('VGG16')
    else:
        raise('Unspecified model.')
    
    

In [9]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
def optimizer_peformance(model, learning_rate, batch_size, weight_decay, epochs, N, mode):
    net = define_models(model)
    net = net.to(device)
    if device == 'cuda':
        net = torch.nn.DataParallel(net)
        cudnn.benchmark = True
        
    train_err = []
    train_loss = []
    test_err = []
    
    opti_name = mode
    optimizer = optimizers(net, opti_name, learning_rate, weight_decay)
    
    lambda_lr = lambda epoch: 0.1 ** (epoch // 80) 
    scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_lr)
    
    start_time = time.time()
    print("-"*30+" Mode "+ mode+" starts" )
    for epoch in range(epochs):
        train_err_i, train_loss_i = train(net, optimizer, epoch)
        train_err.append(train_err_i)
        train_loss.append(train_loss_i)
        test_err.append(test(net))
        scheduler.step()
        print ("--- %s seconds ---" % (time.time() - start_time))
        
    model_name = model + '_' + mode + '_LR'+str(learning_rate) + '_B'+str(batch_size) + '_N'+ str(N) 
    save_err({mode:train_loss}, {mode:train_err}, {mode:test_err}, model+'_'+mode, learning_rate, batch_size, weight_decay, epochs, N)
    return train_loss, train_err, test_err

In [11]:
def optimizer_performance_comparison(model, batch_size, weight_decay, epochs, N):
    train_loss, train_err, test_err = {}, {}, {}
    mode_list = ['Adai LR=1','SGDM','Adai LR=0.1', 'Adam', 'AMSGrad', 'AdamW'] 
    lr_dict = [1., 0.1, 0.1, 1e-4, 1e-4, 1e-4]
    for i,mode in enumerate(mode_list):
        train_loss[mode], train_err[mode], test_err[mode] = optimizer_peformance(model, lr_dict[i], batch_size, weight_decay, epochs, N, mode.split(None, 1)[0])
    return train_loss, train_err, test_err 
    

In [12]:
def plot_figure(model, train_loss, train_err, test_err, batch_size, weight_decay, epochs, N): 
    figure_name = model + '_B'+str(batch_size) + '_N'+ str(N) + '_E' + str(epochs)
    
    plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
    plt.rcParams['image.interpolation'] = 'nearest'
    plt.rcParams['image.cmap'] = 'gray'

    fig = plt.figure()

    axes = plt.gca()
    for key in test_err:
        break
    axes.set_ylim([0., 0.2])
    axes.set_xlim([0,epochs])
    mode_list = ['Adai LR=1','SGDM','Adai LR=0.1', 'Adam', 'AMSGrad', 'AdamW'] 
    colors = ['red','blue','green','orange','pink','cyan','brown','yellow','black']
    for idx,mode in enumerate(mode_list):
        plt.plot(np.arange(1,epochs+1), test_err[mode], label=mode, ls='solid', linewidth=2, color=colors[idx])
        
    plt.ylabel('Test Error')
    plt.xlabel('Epochs')
    plt.grid()
    plt.legend()
    plt.show()
    
    fig.savefig('Results/Test_errors_'+figure_name + '.png')
    fig.savefig('Results/Test_errors_'+figure_name+'.pdf', format='pdf', bbox_inches = 'tight')
    
    fig = plt.figure()
    axes = plt.gca()
    axes.set_yscale('log')
    axes.set_ylim([1e-4, 1.])
    axes.set_xlim([0,epochs])
    for idx,mode in enumerate(mode_list):
        plt.plot(np.arange(1,epochs+1), train_loss[mode], label=mode, ls='solid', linewidth=2, color=colors[idx])

    plt.ylabel('Training Loss')
    plt.xlabel('Epochs')
    plt.grid()
    plt.legend()
    plt.show()
    
    fig.savefig('Results/Training_loss_'+figure_name + '.png')
    fig.savefig('Results/Training_loss_'+figure_name+'.pdf', format='pdf', bbox_inches = 'tight')
    
    
    


In [13]:
def save_err(train_loss, train_err, test_err, model, learning_rate, batch_size, weight_decay, epochs, N):
    csvname = model + '_LR'+str(learning_rate) + '_B'+str(batch_size) + '_N'+ str(N) + '_E' + str(epochs)
    csvname = 'Curves_' + csvname
    csvname ='Results/'+ csvname

    current_name = csvname +'.csv'
    files_present = glob.glob(current_name)
    if files_present:
        print('WARNING: This file already exists!')
    data_dict = {}
    for mode in test_err:
        data_dict[mode+'_test_err'] = test_err[mode]
        data_dict[mode+'_training_err'] = train_err[mode]
        data_dict[mode+'_training_loss'] = train_loss[mode]
    df = pd.DataFrame(data=data_dict)
    if not files_present:
        df.to_csv(current_name, sep=',', header=True, index=False)
    else:
        print('WARNING: This file already exists!')
        for i in range(1,30):
            files_present = glob.glob(csvname+'_'+str(i)+'.csv')
            if not files_present:
                df.to_csv(csvname+'_'+str(i)+'.csv', sep=',', header=True, index=False)
                return None
    return None
    

In [14]:
batch_size = 128
weight_decay = 5e-4
epochs = 200

#Training data size
N = 50000 

In [15]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

In [16]:
model = 'resnet18'

In [ ]:
train_loss, train_err, test_err = optimizer_performance_comparison(model, batch_size, weight_decay, epochs, N)

In [ ]:
plot_figure(model, train_loss, train_err, test_err, batch_size, weight_decay, epochs, N)

In [ ]:
model = 'vgg16'

In [ ]:
train_loss, train_err, test_err = optimizer_performance_comparison(model, batch_size, weight_decay, epochs, N)

In [ ]:
plot_figure(model, train_loss, train_err, test_err, batch_size, weight_decay, epochs, N)